In this script we are going to take 3 datasets: the frequency analysis we computed last week, a dataset with socioeconomic data, and data from the linguisitc inquiry and word count (LIWC) and:

1. merge these 3 datasets ( using year and text name as IDs)
2. create a dataset aggregated by year.
3. create a dataset aggregated by year including all time lags for socioeconomic data. Matching each frequency value at time T with socioeconomic values from time T-20 to T+20, so both in the past and in the future.

In [ ]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import math
import os
import csv

In [ ]:
### READ CSVS AS DATA FRAMEC
base_dir1 = os.getcwd()


CustomFrequencies = pd.read_csv(base_dir1 +'/2.1_output_Frequencies_zscore.csv')

SOC_ECON_data  = pd.read_csv(base_dir1 +'/demographics.csv')

LIWC  = pd.read_csv(base_dir1 +'/LIWC.csv')



In [ ]:
CustomFrequencies['year'] = CustomFrequencies['year'].apply(round)
CustomFrequencies

,Unnamed: 0,Unnamed: 0.1,file,author,year,title,impulsive_feelings,slow_feelings,pleasure,asceticism,...,ratio_behavior_Out,ratio_global_Out,ratio_feelings2,ratio_traits2,ratio_behavior2,ratio_global2,ratio_feelings2_Out,ratio_traits2_Out,ratio_behavior2_Out,ratio_global2_Out
0,0,0,CumberlandRichard-1772-The fashionable love68.txt,CumberlandRichard,1772,The fashionable love68.txt,0.006198,0.011511,0.003896,0.003896,...,2.442308,1.695896,0.124267,1.091658,1.670347,1.354058,0.124267,1.091658,1.670347,1.354058
1,1,1,Lyly John-1591-Mother Bombi.txt,Lyly John,1591,Mother Bombi.txt,0.025325,0.008740,0.010533,0.000896,...,-1.234420,-2.149780,NaN,-2.062751,-0.839139,-2.778762,NaN,-2.062751,-0.839139,-2.778762
2,2,2,WilliamShakespeare-1602-All's Well That Ends.txt,WilliamShakespeare,1602,All's Well That Ends.txt,0.008546,0.014357,0.011451,0.003247,...,0.008390,0.505946,-0.041240,1.771751,-0.005041,0.656877,-0.041240,1.771751,-0.005041,0.656877
3,3,3,HolcroftThomas-1781-Duplicity a comedy159.txt,HolcroftThomas,1781,Duplicity a comedy159.txt,0.013046,0.022831,0.004349,0.001305,...,0.094201,-0.063679,0.048868,-1.215984,0.479909,-0.150461,0.048868,-1.215984,0.479909,-0.150461
4,4,4,CoryeJohn-1672-The generous enemies49.txt,CoryeJohn,1672,The generous enemies49.txt,0.020091,0.022934,0.010614,0.004549,...,0.698000,0.415809,-1.586731,0.023766,0.705725,0.676229,-1.586731,0.023766,0.705725,0.676229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,927,927,Lyly John-1584-Gallathea.txt,Lyly John,1584,Gallathea.txt,0.017418,0.017141,0.034283,0.019353,...,1.228485,-1.705125,-1.742750,NaN,NaN,NaN,-1.742750,NaN,NaN,NaN
928,928,928,anon-1601-The Contention between Liberality an...,anon,1601,The Contention between Liberality and Prodigal...,0.006706,0.015779,0.012229,0.000789,...,-1.389884,0.015454,0.581492,0.845497,-1.198858,-0.198120,0.581492,0.845497,-1.198858,-0.198120
929,929,929,TownleyJames-1780-High life below stai38.txt,TownleyJames,1780,High life below stai38.txt,0.004953,0.013507,0.005403,0.000000,...,-1.918462,1.440611,0.685399,NaN,-0.272369,1.133699,0.685399,NaN,-0.272369,1.133699
930,930,930,Brome Richard-1635-The Lovesick Court or The A...,Brome Richard,1635,The Lovesick Court or The Ambitious Politi.txt,0.010582,0.027827,0.013325,0.002939,...,-0.342280,0.052943,1.300245,0.103390,-0.483466,0.088210,1.300245,0.103390,-0.483466,0.088210


In [ ]:
## Set the period you are intereted in. In this case there are few plays before 1630 and after 1800,
## for most years outside of thie period there are no plays

#CustomFrequencies = CustomFrequencies[(CustomFrequencies['year']>1630) & (CustomFrequencies['year']<1800)]

In [ ]:
SOC_ECON_data['year'] = SOC_ECON_data['year'].apply(round)
SOC_ECON_data

,year,GDPpc,Wages,Books,inflation,life_expectancy,period,marriage,natural pop increase,death rate,...,Migration,Intrinsic growth,net reproduction rate,Gross reproduction rate,annual growth rate,Population,TotalEduMen,TotalEduWomen,TotalEduAll,birth_per_marriage
0,1564,1179,2.10,18.7,-0.238013,38.466,pre_civil_war,9.938,10.108,25.206,...,4762.6,0.934,1.352,2.416,0.860,3118382.0,1.04,0.46,0.75,3.554035
1,1565,1176,1.91,27.9,10.343658,39.068,pre_civil_war,9.794,10.024,24.708,...,4822.8,0.922,1.346,2.368,0.850,3145947.0,1.04,0.46,0.75,3.547070
2,1566,1315,2.12,29.0,-10.279028,39.670,pre_civil_war,9.650,9.940,24.210,...,4883.0,0.910,1.340,2.320,0.840,3173512.0,1.05,0.46,0.75,3.539896
3,1567,1229,2.13,28.3,-0.490940,39.948,pre_civil_war,9.694,9.880,23.914,...,4909.6,0.882,1.328,2.286,0.836,3200853.4,1.05,0.46,0.75,3.486899
4,1568,1222,1.97,23.4,8.183530,40.226,pre_civil_war,9.738,9.820,23.618,...,4936.2,0.854,1.316,2.252,0.832,3228194.8,1.05,0.46,0.76,3.434381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,1806,2062,3.24,84.2,-6.362153,40.580,NaN,8.080,14.220,23.680,...,9422.0,1.520,1.590,2.670,1.320,9232494.0,1.59,1.16,1.38,4.690594
243,1807,2167,3.39,80.6,-4.529617,40.714,NaN,8.148,14.562,23.594,...,9578.2,1.554,1.606,2.690,1.354,9358786.2,1.59,1.16,1.37,4.682867
244,1808,2039,3.20,89.1,6.204380,40.848,NaN,8.216,14.904,23.508,...,9734.4,1.588,1.622,2.710,1.388,9485078.4,1.58,1.16,1.37,4.675268
245,1809,2076,2.84,85.4,12.628866,40.982,NaN,8.284,15.246,23.422,...,9890.6,1.622,1.638,2.730,1.422,9611370.6,1.56,1.14,1.35,4.667793


In [ ]:
LIWC

,Filename,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,AddisonJoseph-1713-Cato A tragedy As 3.txt,1,15226,57.46,82.59,27.09,58.95,15226,11.72,82.98,...,0,0,0,0,0,0,0,0,0,0.02
1,AddisonJoseph-1715-The drummer or the2.txt,1,17155,35.16,84.95,18.45,71.41,17155,9.94,88.52,...,0,0,0,0,0,0,0,0,0,0.00
2,AmesRichard-1691-The siege and surren2.txt,1,7151,69.66,79.49,32.92,39.79,7151,14.14,83.64,...,0,0,0,0,0,0,0,0,0,0.00
3,AndrewsMiles Peter-1795-The mysteries of the4.txt,1,18258,53.26,72.94,36.67,50.95,18258,12.93,85.01,...,0,0,0,0,0,0,0,0,0,0.03
4,anon-1589-King Leir and his Three Daughter.txt,1,21235,30.46,76.65,38.61,76.44,21235,9.55,88.52,...,0,0,0,0,0,0,0,0,0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,WycherleyWilliam-1675-The countrywife a c337.txt,1,28690,26.33,87.22,20.01,79.03,28690,9.47,89.10,...,0,0,0,0,0,0,0,0,0,0.00
928,WycherleyWilliam-1677-The plaindealer a c340.txt,1,37359,32.66,85.20,25.31,65.88,37359,10.65,88.19,...,0,0,0,0,0,0,0,0,0,0.00
929,WycherleyWilliam-1693-The gentleman dancin338.txt,1,30193,22.71,83.15,30.61,70.97,30193,9.88,87.48,...,0,0,0,0,0,0,0,0,0,0.00
930,Yarington Robert-1594-Two Lamentable Tragedies...,1,18264,36.45,73.61,28.38,36.56,18264,10.76,87.08,...,0,0,0,0,0,0,0,0,0,0.05


In [ ]:
## function to add the corresponding socioeconomic value
## it takes a list of years, a particular socioeconomic variable, a column with frequencies, a column with economic data

def add_soc (year,var, freq_data, econ_data):

    ## creates a list of values of a particular socio-economic variable per year
    var_year = list(econ_data[var])

    ## creates a dict with that socio-econ var by year    
    dictionary_var = dict(zip(year, var_year))
    #print(dictionary_var)

    ## creates a list of socioecon values ordered by the position of each fiction text - in freq_text
    var_list = []

    ## it takes the order of years in the text frequency data, and adds it to a new list
    for item in freq_data['year']:    
        #print(item)
        var_list += [dictionary_var[item]]
    return(var_list)

In [ ]:
## add the corresponding socioeconomic values

## make a copy of the data set for higiene
Freq_and_SOC = CustomFrequencies.copy()

## creates a list of years
year = list(SOC_ECON_data['year'])   

## creates a list of all socioecomic variables
list_of_vars = list(SOC_ECON_data.columns)

## iterate over all soc econ vars
for var in list_of_vars:
    try:
        ## creates a new column in our frequency dataframe
        Freq_and_SOC[var] = add_soc(year, var, CustomFrequencies,SOC_ECON_data)
    except KeyError:
        Freq_and_SOC[var] = add_soc(year, var, CustomFrequencies,'')



In [ ]:
Freq_and_SOC

,Unnamed: 0,Unnamed: 0.1,file,author,year,title,impulsive_feelings,slow_feelings,pleasure,asceticism,...,Migration,Intrinsic growth,net reproduction rate,Gross reproduction rate,annual growth rate,Population,TotalEduMen,TotalEduWomen,TotalEduAll,birth_per_marriage
0,0,0,CumberlandRichard-1772-The fashionable love68.txt,CumberlandRichard,1772,The fashionable love68.txt,0.006198,0.011511,0.003896,0.003896,...,5875.8,1.006,1.358,2.392,0.854,6681381.2,1.80,1.10,1.45,4.117398
1,1,1,Lyly John-1591-Mother Bombi.txt,Lyly John,1591,Mother Bombi.txt,0.025325,0.008740,0.010533,0.000896,...,5676.0,0.810,1.300,2.350,0.590,3937846.0,1.15,0.49,0.82,3.427962
2,2,2,WilliamShakespeare-1602-All's Well That Ends.txt,WilliamShakespeare,1602,All's Well That Ends.txt,0.008546,0.014357,0.011451,0.003247,...,6160.8,0.752,1.272,2.260,0.712,4191511.2,1.22,0.53,0.87,3.669540
3,3,3,HolcroftThomas-1781-Duplicity a comedy159.txt,HolcroftThomas,1781,Duplicity a comedy159.txt,0.013046,0.022831,0.004349,0.001305,...,6027.0,0.760,1.260,2.400,0.620,7206139.0,1.76,1.12,1.44,4.067678
4,4,4,CoryeJohn-1672-The generous enemies49.txt,CoryeJohn,1672,The generous enemies49.txt,0.020091,0.022934,0.010614,0.004549,...,5924.6,-0.110,0.968,1.822,0.022,5164048.8,1.64,0.79,1.21,4.185316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,927,927,Lyly John-1584-Gallathea.txt,Lyly John,1584,Gallathea.txt,0.017418,0.017141,0.034283,0.019353,...,5257.8,0.958,1.372,2.384,0.748,3756963.8,1.10,0.48,0.79,3.472140
928,928,928,anon-1601-The Contention between Liberality an...,anon,1601,The Contention between Liberality and Prodigal...,0.006706,0.015779,0.012229,0.000789,...,6138.0,0.750,1.270,2.270,0.700,4161784.0,1.21,0.53,0.87,3.644737
929,929,929,TownleyJames-1780-High life below stai38.txt,TownleyJames,1780,High life below stai38.txt,0.004953,0.013507,0.005403,0.000000,...,6094.2,0.806,1.278,2.408,0.662,7147606.0,1.78,1.12,1.45,4.079162
930,930,930,Brome Richard-1635-The Lovesick Court or The A...,Brome Richard,1635,The Lovesick Court or The Ambitious Politi.txt,0.010582,0.027827,0.013325,0.002939,...,6648.0,0.286,1.100,2.148,0.258,5057125.2,1.32,0.61,0.97,3.748275


In [ ]:
## function to add data from Linguistic Inquiry and Word count (LIWC)
## here we identify the rows by text name

def add_LIWC (var, freq_data, LIWC_data):
    LIWC_list = []
    print(var)

    ## iterate over files in our frequency dataframe
    for index1, rowz in freq_data.iterrows():
        LIWC_concept= np.nan

        text_code = rowz['file']
        #print(text_code)
        
        ## iterates over files in LIWC dataframe
        for index2, rowz2 in LIWC_data.iterrows():
            text_code2 = rowz2['Filename']
            #print(text_code2)

            ## if name is the same
            if text_code == text_code2:
                #print(text_code)
                
                ## identify the row
                LIWC_concept = rowz2[var]
                
        ## and add it to a lisst
        LIWC_list += [LIWC_concept]
    return (LIWC_list)


In [ ]:
## Add data from Linguistic Inquiry and Word count (LIWC)

Freq_SOC_and_LIWC = Freq_and_SOC

LIWC_vars = ['family','friend','sexual','focusfuture','focuspresent']

for var in LIWC_vars:
    var_name = var + 'LIWC'
    Freq_SOC_and_LIWC[var_name]=add_LIWC (var, CustomFrequencies, LIWC)
    

family
friend
sexual
focusfuture
focuspresent


In [ ]:
Freq_SOC_and_LIWC

,Unnamed: 0,Unnamed: 0.1,file,author,year,title,impulsive_feelings,slow_feelings,pleasure,asceticism,...,Population,TotalEduMen,TotalEduWomen,TotalEduAll,birth_per_marriage,familyLIWC,friendLIWC,sexualLIWC,focusfutureLIWC,focuspresentLIWC
0,0,0,CumberlandRichard-1772-The fashionable love68.txt,CumberlandRichard,1772,The fashionable love68.txt,0.006198,0.011511,0.003896,0.003896,...,6681381.2,1.80,1.10,1.45,4.117398,0.68,0.46,0.15,2.03,13.68
1,1,1,Lyly John-1591-Mother Bombi.txt,Lyly John,1591,Mother Bombi.txt,0.025325,0.008740,0.010533,0.000896,...,3937846.0,1.15,0.49,0.82,3.427962,1.73,0.17,0.11,3.44,13.88
2,2,2,WilliamShakespeare-1602-All's Well That Ends.txt,WilliamShakespeare,1602,All's Well That Ends.txt,0.008546,0.014357,0.011451,0.003247,...,4191511.2,1.22,0.53,0.87,3.669540,0.92,0.32,0.21,2.44,12.84
3,3,3,HolcroftThomas-1781-Duplicity a comedy159.txt,HolcroftThomas,1781,Duplicity a comedy159.txt,0.013046,0.022831,0.004349,0.001305,...,7206139.0,1.76,1.12,1.44,4.067678,0.64,0.47,0.13,2.04,13.53
4,4,4,CoryeJohn-1672-The generous enemies49.txt,CoryeJohn,1672,The generous enemies49.txt,0.020091,0.022934,0.010614,0.004549,...,5164048.8,1.64,0.79,1.21,4.185316,0.79,0.31,0.17,2.33,12.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,927,927,Lyly John-1584-Gallathea.txt,Lyly John,1584,Gallathea.txt,0.017418,0.017141,0.034283,0.019353,...,3756963.8,1.10,0.48,0.79,3.472140,0.58,0.17,0.49,3.66,13.84
928,928,928,anon-1601-The Contention between Liberality an...,anon,1601,The Contention between Liberality and Prodigal...,0.006706,0.015779,0.012229,0.000789,...,4161784.0,1.21,0.53,0.87,3.644737,0.38,0.42,0.29,3.53,13.38
929,929,929,TownleyJames-1780-High life below stai38.txt,TownleyJames,1780,High life below stai38.txt,0.004953,0.013507,0.005403,0.000000,...,7147606.0,1.78,1.12,1.45,4.079162,0.27,0.52,0.07,2.87,13.51
930,930,930,Brome Richard-1635-The Lovesick Court or The A...,Brome Richard,1635,The Lovesick Court or The Ambitious Politi.txt,0.010582,0.027827,0.013325,0.002939,...,5057125.2,1.32,0.61,0.97,3.748275,1.09,0.29,0.19,3.12,13.52


In [ ]:
LIWC_vars2 = []
for item in LIWC_vars:
    LIWC_vars2 += [item+ 'LIWC']


In [ ]:
Freq_SOC_and_LIWC2 = Freq_SOC_and_LIWC.copy()

### z-score sociodemographic variables
from scipy import stats

list_of_vars = list(SOC_ECON_data.columns) + LIWC_vars2

for var in list_of_vars:
    try:
        var_z = 'z_' + var
        Freq_SOC_and_LIWC2[var_z] = stats.zscore(Freq_SOC_and_LIWC2[var], nan_policy='omit')
    except TypeError:
        continue

Freq_SOC_and_LIWC2

,Unnamed: 0,Unnamed: 0.1,file,author,year,title,impulsive_feelings,slow_feelings,pleasure,asceticism,...,z_Population,z_TotalEduMen,z_TotalEduWomen,z_TotalEduAll,z_birth_per_marriage,z_familyLIWC,z_friendLIWC,z_sexualLIWC,z_focusfutureLIWC,z_focuspresentLIWC
0,0,0,CumberlandRichard-1772-The fashionable love68.txt,CumberlandRichard,1772,The fashionable love68.txt,0.006198,0.011511,0.003896,0.003896,...,1.114974,0.922204,1.323815,1.143754,0.595547,-0.173500,0.390080,-0.389555,-1.093526,0.758429
1,1,1,Lyly John-1591-Mother Bombi.txt,Lyly John,1591,Mother Bombi.txt,0.025325,0.008740,0.010533,0.000896,...,-1.315381,-1.590597,-1.389289,-1.540794,-2.069886,2.664321,-1.299247,-0.730301,1.373214,0.867539
2,2,2,WilliamShakespeare-1602-All's Well That Ends.txt,WilliamShakespeare,1602,All's Well That Ends.txt,0.008546,0.014357,0.011451,0.003247,...,-1.090672,-1.319988,-1.211381,-1.327735,-1.135919,0.475145,-0.425457,0.121565,-0.376247,0.300169
3,3,3,HolcroftThomas-1781-Duplicity a comedy159.txt,HolcroftThomas,1781,Duplicity a comedy159.txt,0.013046,0.022831,0.004349,0.001305,...,1.579830,0.767570,1.412769,1.101142,0.403323,-0.281607,0.448333,-0.559928,-1.076031,0.676597
4,4,4,CoryeJohn-1672-The generous enemies49.txt,CoryeJohn,1672,The generous enemies49.txt,0.020091,0.022934,0.010614,0.004549,...,-0.229152,0.303668,-0.054976,0.121069,0.858122,0.123796,-0.483710,-0.219182,-0.568687,0.332902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,927,927,Lyly John-1584-Gallathea.txt,Lyly John,1584,Gallathea.txt,0.017418,0.017141,0.034283,0.019353,...,-1.475615,-1.783889,-1.433767,-1.668630,-1.899088,-0.443768,-1.299247,2.506789,1.758096,0.845717
928,928,928,anon-1601-The Contention between Liberality an...,anon,1601,The Contention between Liberality and Prodigal...,0.006706,0.015779,0.012229,0.000789,...,-1.117006,-1.358646,-1.211381,-1.327735,-1.231811,-0.984306,0.157070,0.803057,1.530666,0.594765
929,929,929,TownleyJames-1780-High life below stai38.txt,TownleyJames,1780,High life below stai38.txt,0.004953,0.013507,0.005403,0.000000,...,1.527978,0.844887,1.412769,1.143754,0.447721,-1.281601,0.739596,-1.071048,0.376021,0.665686
930,930,930,Brome Richard-1635-The Lovesick Court or The A...,Brome Richard,1635,The Lovesick Court or The Ambitious Politi.txt,0.010582,0.027827,0.013325,0.002939,...,-0.323870,-0.933403,-0.855564,-0.901616,-0.831522,0.934602,-0.600215,-0.048809,0.813387,0.671142


In [ ]:
## Create LIWC ratios for external validation

Freq_SOC_and_LIWC2['family.to.sexual'] = Freq_SOC_and_LIWC2['z_familyLIWC'] - Freq_SOC_and_LIWC2['z_sexualLIWC']
Freq_SOC_and_LIWC2['friend.to.sexual'] = Freq_SOC_and_LIWC2['z_friendLIWC'] - Freq_SOC_and_LIWC2['z_sexualLIWC']
Freq_SOC_and_LIWC2['future.to.present'] = Freq_SOC_and_LIWC2['z_focusfutureLIWC'] - Freq_SOC_and_LIWC2['z_focuspresentLIWC']



In [ ]:
### SAVE LIST AS CSV ###

Freq_SOC_and_LIWC2.to_csv('3.1_output_CompleteDataset.csv') 

## In this section we create a dataset aggregated by year

In [ ]:

##list of variables to be aggregated
print(list(Freq_SOC_and_LIWC2.iloc[:,3:]))

## Then we group them by year and save them
year_data=Freq_SOC_and_LIWC.iloc[:,3:].groupby('year').mean()

['author', 'year', 'title', 'impulsive_feelings', 'slow_feelings', 'pleasure', 'asceticism', 'short_term', 'long_term', 'physical', 'moral', 'other_body', 'other_mental', 'fast_all', 'slow_all', 'ratio_feelings', 'ratio_traits', 'ratio_behavior', 'ratio_global', 'z_impulsive_feelings', 'z_slow_feelings', 'z_pleasure', 'z_asceticism', 'z_short_term', 'z_long_term', 'z_physical', 'z_moral', 'z_other_body', 'z_other_mental', 'z_fast_all', 'z_slow_all', 'z_ratio_feelings', 'z_ratio_traits', 'z_ratio_behavior', 'z_ratio_global', 'impulsive_feelings_Out', 'slow_feelings_Out', 'pleasure_Out', 'asceticism_Out', 'short_term_Out', 'long_term_Out', 'physical_Out', 'moral_Out', 'other_body_Out', 'other_mental_Out', 'fast_all_Out', 'slow_all_Out', 'ratio_feelings_Out', 'ratio_traits_Out', 'ratio_behavior_Out', 'ratio_global_Out', 'ratio_feelings2', 'ratio_traits2', 'ratio_behavior2', 'ratio_global2', 'ratio_feelings2_Out', 'ratio_traits2_Out', 'ratio_behavior2_Out', 'ratio_global2_Out', 'GDPpc', 'W

In [ ]:
## Aggregated dataset
year_data2=year_data.reset_index()
year_data2

,year,impulsive_feelings,slow_feelings,pleasure,asceticism,short_term,long_term,physical,moral,other_body,...,Population,TotalEduMen,TotalEduWomen,TotalEduAll,birth_per_marriage,familyLIWC,friendLIWC,sexualLIWC,focusfutureLIWC,focuspresentLIWC
0,1576,0.010775,0.020676,0.012522,0.004659,0.001747,0.027373,0.008154,0.006989,0.013395,...,3447944.0,1.09,0.47,0.78,3.461538,0.500000,0.490000,0.100000,4.230000,15.270000
1,1577,0.006369,0.020170,0.019904,0.000531,0.006104,0.017251,0.010350,0.008227,0.013004,...,3484643.6,1.09,0.47,0.78,3.488875,0.930000,0.370000,0.120000,3.460000,15.740000
2,1583,0.010438,0.012228,0.019684,0.003579,0.005070,0.005368,0.011631,0.013719,0.036684,...,3715123.2,1.10,0.48,0.79,3.515727,0.250000,0.130000,0.090000,3.150000,13.690000
3,1584,0.012301,0.012892,0.034451,0.007515,0.006270,0.008520,0.032917,0.013697,0.024115,...,3756963.8,1.10,0.48,0.79,3.472140,0.356667,0.176667,0.276667,3.210000,12.443333
4,1587,0.008112,0.016726,0.012629,0.001505,0.002425,0.009033,0.009451,0.011374,0.017144,...,3860085.2,1.12,0.48,0.80,3.393529,0.672500,0.307500,0.115000,2.905000,10.727500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,1796,0.012578,0.022074,0.008726,0.003863,0.004029,0.009712,0.008111,0.018394,0.015773,...,8256323.0,1.65,1.13,1.39,4.335775,0.616667,0.486667,0.331667,2.121667,11.211667
182,1797,0.007702,0.025973,0.008210,0.003014,0.004279,0.012269,0.010895,0.016683,0.013689,...,8339346.2,1.65,1.12,1.39,4.312050,0.662500,0.512500,0.100000,2.625000,12.777500
183,1798,0.008492,0.025373,0.009031,0.002570,0.002970,0.021091,0.012515,0.011256,0.027098,...,8422369.4,1.66,1.14,1.40,4.289120,1.001429,0.635714,0.121429,2.657143,12.532857
184,1799,0.005459,0.024216,0.009707,0.003341,0.002859,0.014801,0.010265,0.013575,0.015333,...,8505392.6,1.65,1.14,1.39,4.266945,0.778333,0.478333,0.123333,2.230000,11.190000


In [ ]:
## here we count the number of occurences per year and add them to the dataset

counts_per_year =[]

for year in year_data2['year']:
    counts = Freq_SOC_and_LIWC2[Freq_SOC_and_LIWC2["year"]==year].count()["file"]
    counts_per_year += [counts]
    print(year,counts)

year_data2.insert(1,'text_per_year',counts_per_year)

1576 1
1577 1
1583 1
1584 3
1587 4
1588 4
1589 3
1590 7
1591 7
1592 7
1593 4
1594 11
1595 4
1596 4
1597 4
1598 3
1599 11
1600 7
1601 6
1602 10
1603 3
1604 10
1605 14
1606 10
1607 12
1608 12
1609 3
1610 5
1611 11
1612 7
1613 6
1614 5
1615 1
1616 4
1617 4
1618 4
1619 8
1620 8
1621 7
1622 7
1623 3
1624 6
1625 3
1626 6
1627 2
1628 4
1629 4
1630 3
1631 9
1632 9
1633 7
1634 5
1635 8
1636 4
1637 6
1638 11
1639 7
1640 11
1641 4
1642 3
1652 1
1657 1
1659 2
1660 4
1661 5
1662 2
1663 7
1664 10
1665 3
1666 1
1667 3
1668 9
1669 5
1670 3
1671 11
1672 5
1673 7
1674 2
1675 8
1676 6
1677 16
1678 11
1679 9
1680 12
1681 7
1682 11
1683 2
1684 5
1685 4
1686 1
1687 4
1688 3
1689 3
1690 9
1691 8
1692 7
1693 9
1694 7
1695 3
1696 17
1697 11
1698 12
1699 5
1700 10
1701 1
1702 1
1703 2
1704 2
1706 4
1707 1
1709 3
1710 1
1711 1
1713 1
1714 3
1715 4
1717 2
1718 1
1721 2
1722 1
1723 2
1724 3
1728 1
1730 2
1731 1
1734 3
1737 1
1738 1
1739 1
1741 1
1742 1
1743 1
1745 2
1746 1
1747 2
1749 3
1751 1
1752 3
1753 4
1754 3

In [ ]:
year_data2

,year,text_per_year,impulsive_feelings,slow_feelings,pleasure,asceticism,short_term,long_term,physical,moral,...,Population,TotalEduMen,TotalEduWomen,TotalEduAll,birth_per_marriage,familyLIWC,friendLIWC,sexualLIWC,focusfutureLIWC,focuspresentLIWC
0,1576,1,0.010775,0.020676,0.012522,0.004659,0.001747,0.027373,0.008154,0.006989,...,3447944.0,1.09,0.47,0.78,3.461538,0.500000,0.490000,0.100000,4.230000,15.270000
1,1577,1,0.006369,0.020170,0.019904,0.000531,0.006104,0.017251,0.010350,0.008227,...,3484643.6,1.09,0.47,0.78,3.488875,0.930000,0.370000,0.120000,3.460000,15.740000
2,1583,1,0.010438,0.012228,0.019684,0.003579,0.005070,0.005368,0.011631,0.013719,...,3715123.2,1.10,0.48,0.79,3.515727,0.250000,0.130000,0.090000,3.150000,13.690000
3,1584,3,0.012301,0.012892,0.034451,0.007515,0.006270,0.008520,0.032917,0.013697,...,3756963.8,1.10,0.48,0.79,3.472140,0.356667,0.176667,0.276667,3.210000,12.443333
4,1587,4,0.008112,0.016726,0.012629,0.001505,0.002425,0.009033,0.009451,0.011374,...,3860085.2,1.12,0.48,0.80,3.393529,0.672500,0.307500,0.115000,2.905000,10.727500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,1796,6,0.012578,0.022074,0.008726,0.003863,0.004029,0.009712,0.008111,0.018394,...,8256323.0,1.65,1.13,1.39,4.335775,0.616667,0.486667,0.331667,2.121667,11.211667
182,1797,4,0.007702,0.025973,0.008210,0.003014,0.004279,0.012269,0.010895,0.016683,...,8339346.2,1.65,1.12,1.39,4.312050,0.662500,0.512500,0.100000,2.625000,12.777500
183,1798,7,0.008492,0.025373,0.009031,0.002570,0.002970,0.021091,0.012515,0.011256,...,8422369.4,1.66,1.14,1.40,4.289120,1.001429,0.635714,0.121429,2.657143,12.532857
184,1799,6,0.005459,0.024216,0.009707,0.003341,0.002859,0.014801,0.010265,0.013575,...,8505392.6,1.65,1.14,1.39,4.266945,0.778333,0.478333,0.123333,2.230000,11.190000


In [ ]:

### SAVE year data AS CSV ###
year_data2.to_csv('3.2_output_CompleteData_by_year.csv') 

### The last step is to add the time lags of socio-economic variables, we basically create new columns where each socioeconomic variable is lagged, e.g. the frequency data of 1800 is paired with the row of the frequency data of 1801, 1802, 1803, but also 1799, 1798, etc.

In [ ]:
year_data3 = year_data2.copy()

lag_vars= list(SOC_ECON_data.columns)
year = list(SOC_ECON_data['year'][1:])
#print(SOC_ECON_data)

## this loop takes each socioeconomic variable
for var in lag_vars:
    var_list = list(SOC_ECON_data[var][1:])
    dictionary_var = dict(zip(year, var_list))

    ## and creates the lags within T-20 and T+20
    for lag in range(-20,21):
        var_lag_list = []

        ## for each row we find what the lagged year would be
        for index1, row in year_data3.iterrows():
            yr_df = row['year']+lag
            #print(yr_df)
            try:

                ##and find what the socioeconomic value was in the lagged year
                var1 = dictionary_var[yr_df]
                var_lag_list +=[var1]
            except KeyError:
                var_lag_list +=[np.nan]
            
        #print(GDP_lag_list)
        year_data3[var+str(lag)]=var_lag_list
year_data3

,year,text_per_year,impulsive_feelings,slow_feelings,pleasure,asceticism,short_term,long_term,physical,moral,...,birth_per_marriage11,birth_per_marriage12,birth_per_marriage13,birth_per_marriage14,birth_per_marriage15,birth_per_marriage16,birth_per_marriage17,birth_per_marriage18,birth_per_marriage19,birth_per_marriage20
0,1576,1,0.010775,0.020676,0.012522,0.004659,0.001747,0.027373,0.008154,0.006989,...,3.393529,3.402077,3.410665,3.419293,3.427962,3.471035,3.515553,3.561589,3.609222,3.658537
1,1577,1,0.006369,0.020170,0.019904,0.000531,0.006104,0.017251,0.010350,0.008227,...,3.402077,3.410665,3.419293,3.427962,3.471035,3.515553,3.561589,3.609222,3.658537,3.655647
2,1583,1,0.010438,0.012228,0.019684,0.003579,0.005070,0.005368,0.011631,0.013719,...,3.561589,3.609222,3.658537,3.655647,3.652825,3.650067,3.647372,3.644737,3.669540,3.694742
3,1584,3,0.012301,0.012892,0.034451,0.007515,0.006270,0.008520,0.032917,0.013697,...,3.609222,3.658537,3.655647,3.652825,3.650067,3.647372,3.644737,3.669540,3.694742,3.720350
4,1587,4,0.008112,0.016726,0.012629,0.001505,0.002425,0.009033,0.009451,0.011374,...,3.652825,3.650067,3.647372,3.644737,3.669540,3.694742,3.720350,3.746377,3.772831,3.752577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,1796,6,0.012578,0.022074,0.008726,0.003863,0.004029,0.009712,0.008111,0.018394,...,4.682867,4.675268,4.667793,4.660441,NaN,NaN,NaN,NaN,NaN,NaN
182,1797,4,0.007702,0.025973,0.008210,0.003014,0.004279,0.012269,0.010895,0.016683,...,4.675268,4.667793,4.660441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,1798,7,0.008492,0.025373,0.009031,0.002570,0.002970,0.021091,0.012515,0.011256,...,4.667793,4.660441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,1799,6,0.005459,0.024216,0.009707,0.003341,0.002859,0.014801,0.010265,0.013575,...,4.660441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
### SAVE year data AS CSV ###

year_data3.to_csv('3.3_output_CompleteData_lags.csv') 